# Toponym Resolution with T-Res

TODO: 
 - briefly explain T-Res.
 - explain what is an HTTP API (and that it relies on the server running).

In [1]:
import requests
from dataclasses import dataclass
from dacite import from_dict

Helper functions for calling the T-Res API and parsing the response:

In [29]:
HOST = "20.0.184.45"
API_URL = f"http://{HOST}:8000/v2/t-res_deezy_reldisamb-wpubl-wmtops"

@dataclass
class Toponym:
    mention: str
    sentence: str
    pos: int
    end_pos: int
    prediction: str
    cross_cand_score: dict
    latlon: list
    wkdt_class: str
    
    def __str__(self):
        s = f"Toponym:\t{self.mention}"
        s += f"\nWikidata ID:\t{self.prediction}"
        s += f"\nCoordinates:\t{self.latlon}"
        s += f"\nLinking score:\t{self.cross_cand_score[self.prediction]}"
        return s

    def __repr__(self):
        return self.__str__()

class Toponyms:
    toponyms: list

    def __init__(self, data):
        if not isinstance(data, list):
            raise ValueError("Toponyms data must be a list.")
        self.toponyms = [from_dict(data_class=Toponym, data=t) for t in data]

    def __str__(self):
        return '\n\n'.join([t.__str__() for t in self.toponyms])

    def __repr__(self):
        return self.__str__()

def validate_query(query):
    if not "text" in query.keys():
        raise ValueError("T-Res API query must contain an item named `text`")
    return

def call_api(query):
    validate_query(query)
    return parse_api_response(requests.get(f'{API_URL}/toponym_resolution', json=query))

def parse_api_response(response):
    if response.status_code != 200:
        print(f"HTTP error code: {response.status_code}")
        print(f"Reason: {response.reason}")
    return Toponyms(response.json())

### Simple example of toponym resolution from text

In [ ]:
query = {"text": "A remarkable case of rattening has just occurred in the building trade at Newtown."}
result = call_api(query)
result

### Example with place of publication information

In [ ]:
query = {
        "text": "A remarkable case of rattening has just occurred in the building trade at Newtown.",
        "place": "Powys",
        "place_wqid": "Q156150"
        }
result = call_api(query)
result

In [ ]:
query = {
        "text": "A remarkable case of rattening has just occurred in the building trade at Newtown. It started in Powys.",
        "place": "Powys",
        "place_wqid": "Q156150"
        }
result = call_api(query)
result